In [ ]:
!pip install silence_tensorflow tensorflow scikit-learn tqdm numpy pandas

### 라이브러리 임포트

In [1]:
from silence_tensorflow import silence_tensorflow
silence_tensorflow()
import warnings
warnings.filterwarnings('ignore')

import os
import numpy as np
import pandas as pd
import tensorflow as tf
import random
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dense, Dropout

# 시드값 고정
def set_seed(seed):
    np.random.seed(seed)
    tf.random.set_seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

set_seed(42)
os.environ["CUDA_VISIBLE_DEVICES"] = '0' # 0번 GPU만 사용 

In [2]:
npy_files = [f for f in os.listdir('./data/train_npy/') if f.endswith('.npy')] # 훈련 npy 파일이 들어있는 폴더 경로

spectrogram_shapes = {}
for file in npy_files:
    data = np.load(f"./data/train_npy/{file}") # 훈련 npy 파일이 들어있는 경로를 {file}앞까지 삽입
    spectrogram_shapes[file] = data.shape

In [3]:
def apply_zero_padding(spectrogram, max_length):
    padding_length = max_length - spectrogram.shape[1]
    if padding_length > 0:
        return np.pad(spectrogram, ((0, 0), (0, padding_length)), 'constant')
    else:
        return spectrogram

In [4]:
# 가장 긴 시간 축의 길이를 찾습니다.
max_time_length = max(shape[1] for shape in spectrogram_shapes.values())

padded_spectrograms = {}
for file, shape in spectrogram_shapes.items():
    spectrogram = np.load(f"./data/train_npy/{file}") # 훈련 npy 파일이 들어있는 폴더 경로
    padded_spectrogram = apply_zero_padding(spectrogram, max_time_length)
    padded_spectrograms[file] = padded_spectrogram

In [5]:
labels_df = pd.read_csv('./data/train.csv') # 정답 레이블이 있는 csv 파일의 경로
labels_dict = labels_df.set_index('file').to_dict()['label']
padded_data_labels = [labels_dict[file] for file in padded_spectrograms.keys()]

padded_data_array = np.array(list(padded_spectrograms.values()), dtype='float32')
padded_data_labels = np.array(padded_data_labels, dtype='int')

In [6]:
X_train, X_val, y_train, y_val = train_test_split(
    padded_data_array,
    padded_data_labels,
    test_size=0.1,
    random_state=42
)

classes = np.unique(y_train)
class_weights = compute_class_weight(class_weight='balanced', classes=classes, y=y_train)
class_weight_dict = {classes[i]: class_weights[i] for i in range(len(classes))}

X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], X_train.shape[2], 1))
X_val = X_val.reshape((X_val.shape[0], X_val.shape[1], X_val.shape[2], 1))

In [7]:
def simpleCNN_1(input_shape):
    model = Sequential([
    Conv2D(16, (3, 3), activation='relu', padding='same', input_shape=input_shape),
    MaxPooling2D(2, 2, padding='same'),
    Conv2D(24, (3, 3), activation='relu', padding='same'),
    MaxPooling2D(2, 2, padding='same'),    
    Conv2D(32, (3, 3), activation='relu', padding='same'),
    MaxPooling2D(2, 2, padding='same'),
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    MaxPooling2D(2, 2, padding='same'),
    GlobalAveragePooling2D(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(56, activation='relu'),
    Dropout(0.5),
    Dense(32, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
    ])
    return model

def simpleCNN_2(input_shape):
    model = Sequential([
    Conv2D(16, (3, 3), activation='relu', padding='same', input_shape=input_shape),
    MaxPooling2D(2, 2, padding='same'),
    Conv2D(24, (3, 3), activation='relu', padding='same'),
    MaxPooling2D(2, 2, padding='same'),    
    Conv2D(32, (3, 3), activation='relu', padding='same'),
    MaxPooling2D(2, 2, padding='same'),
    Conv2D(40, (3, 3), activation='relu', padding='same'),
    MaxPooling2D(2, 2, padding='same'),
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    MaxPooling2D(2, 2, padding='same'),
    GlobalAveragePooling2D(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(56, activation='relu'),
    Dropout(0.5),
    Dense(32, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
    ])
    return model

def simpleCNN_3(input_shape):
    model = Sequential([
    Conv2D(16, (3, 3), activation='relu', padding='same', input_shape=input_shape),
    MaxPooling2D(2, 2, padding='same'),
    Conv2D(24, (3, 3), activation='relu', padding='same'),
    MaxPooling2D(2, 2, padding='same'),    
    Conv2D(32, (3, 3), activation='relu', padding='same'),
    MaxPooling2D(2, 2, padding='same'),
    Conv2D(40, (3, 3), activation='relu', padding='same'),
    MaxPooling2D(2, 2, padding='same'),
    Conv2D(48, (3, 3), activation='relu', padding='same'),
    MaxPooling2D(2, 2, padding='same'),
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    MaxPooling2D(2, 2, padding='same'),
    GlobalAveragePooling2D(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(56, activation='relu'),
    Dropout(0.5),
    Dense(32, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
    ])
    return model

In [8]:
model = simpleCNN_1(input_shape=(X_train.shape[1], X_train.shape[2], 1)) # 앙상블을 사용하기 때문에 3개의 모델을 모두 훈련하여야 합니다.
#model = simpleCNN_2(input_shape=(X_train.shape[1], X_train.shape[2], 1))
#model = simpleCNN_3(input_shape=(X_train.shape[1], X_train.shape[2], 1))

early_stopping = EarlyStopping(
    monitor='val_loss',  
    patience=60,         
    verbose=1,           
    restore_best_weights=True 
)

model_checkpoint = ModelCheckpoint(
    'best_model_v1.h5', # val_loss가 가장 낮은 가중치 파일 저장, 모델 마다 이름을 다르게 저장해야 합니다.
    monitor='val_loss',
    save_best_only=True,
    verbose=1
)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

In [9]:
history = model.fit(
    X_train, y_train,
    epochs=1000,
    batch_size=32,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping, model_checkpoint,
              tf.keras.callbacks.ReduceLROnPlateau(
                  monitor='val_loss',
                  factor=0.8,
                  patience=30,
                  verbose=1
              )],
    class_weight=class_weight_dict
)

Epoch 1/1000
169/169 [==============================] - ETA: 0s - loss: 6.1354 - accuracy: 0.6519
Epoch 1: val_loss improved from inf to 0.67306, saving model to best_model_v1.h5
169/169 [==============================] - 14s 38ms/step - loss: 6.1354 - accuracy: 0.6519 - val_loss: 0.6731 - val_accuracy: 0.7400 - lr: 0.0010
Epoch 2/1000
167/169 [============================>.] - ETA: 0s - loss: 1.5075 - accuracy: 0.6841
Epoch 2: val_loss improved from 0.67306 to 0.65542, saving model to best_model_v1.h5
169/169 [==============================] - 5s 31ms/step - loss: 1.5240 - accuracy: 0.6833 - val_loss: 0.6554 - val_accuracy: 0.7433 - lr: 0.0010
Epoch 3/1000
169/169 [==============================] - ETA: 0s - loss: 0.9203 - accuracy: 0.6780
Epoch 3: val_loss improved from 0.65542 to 0.64106, saving model to best_model_v1.h5
169/169 [==============================] - 4s 27ms/step - loss: 0.9203 - accuracy: 0.6780 - val_loss: 0.6411 - val_accuracy: 0.7467 - lr: 0.0010
Epoch 4/1000
167/16